#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [108]:
import pandas as pd
import pymongo
import datetime

In [109]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['kickstarter']

In [110]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colonnes 6,8,10,12. 

In [112]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Nettoyer les données

In [113]:
# On vérifie que les types de chaque variable sont bien définis.
df_ks.dtypes

ID                    int64
name                 object
category             object
main_category        object
currency             object
deadline             object
goal                 object
launched             object
pledged              object
state                object
backers              object
country              object
usd pledged          object
usd_pledged_real    float64
dtype: object

In [35]:
df_ks["goal"]=df_ks["goal"].astype(float)
# Il y a visiblement une erreur lors de l'entrée des données. Certaines données sont dans les mauvaises colonnes. On peut donc s'attendre à avoir un problème sur chaque colonne.

ValueError: could not convert string to float: '2014-04-17'

In [114]:
df_ks.loc[66141, :]
# Cette ligne a été mal remplie.

ID                                        85964225
name                Debut Album from Michael Adam 
category                          Grace is Leaving
main_category                           Indie Rock
currency                                     Music
deadline                                       USD
goal                                    2014-04-17
launched                                     700.0
pledged                        2014-04-02 21:56:35
state                                        850.0
backers                                 successful
country                                         32
usd pledged                                     US
usd_pledged_real                               850
Name: 66141, dtype: object

In [115]:
# On remet les données dans l'ordre.
dic = [85964225, 'Debut Album from Michael Adam Grace is Leaving', 'Indie Rock', 'Music', 'USD', '2014-04-17', 700.0, '2014-04-02 21:56:35', 850.0, "successful", 32, 'US', 850, 850.00]
df_ks.loc[[66141], :] = dic
df_ks.loc[[66141], :] 

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
66141,85964225,Debut Album from Michael Adam Grace is Leaving,Indie Rock,Music,USD,2014-04-17,700,2014-04-02 21:56:35,850,successful,32,US,850,850.0


In [116]:
df_ks.loc[75119,:]
# On remarque qu'il y a des NaN. 

ID                            387897081
name                 Memory and Weather
category                          Music
main_category                     Music
currency                            USD
deadline                     2015-02-07
goal                             6000.0
launched            2015-01-17 00:56:33
pledged                          6050.0
state                         undefined
backers                               0
country                            N,0"
usd pledged                         NaN
usd_pledged_real                   6050
Name: 75119, dtype: object

In [117]:
df_ks.isna().sum()
# Nous allons conserver seulement les lignes où il n'y a pas de Na.

ID                     0
name                   2
category               0
main_category          0
currency               0
deadline               0
goal                   0
launched               0
pledged                0
state                  0
backers                0
country                0
usd pledged         1482
usd_pledged_real       0
dtype: int64

In [118]:
df_ks = df_ks[df_ks.isna().any(axis=1)==False]
df_ks.isna().sum()

ID                  0
name                0
category            0
main_category       0
currency            0
deadline            0
goal                0
launched            0
pledged             0
state               0
backers             0
country             0
usd pledged         0
usd_pledged_real    0
dtype: int64

In [119]:
# On formate le type de données pour profiter des méthodes implémentées par Mongo.
dtype = {'ID' : 'int64', 'name' : 'string', 'category' : 'string', 'main_category' : 'string', 'currency' : 'string', 'deadline' : 'datetime64[ns]', 'goal' : 'float64', 'launched' : 'datetime64[ns]', 'pledged' : 'float64', 'state' : 'string', 'backers' : 'float64', 'country' : 'string', 'usd pledged' : 'float64', 'usd_pledged_real' : 'float64'}
df_ks= df_ks.astype(dtype)
df_ks.dtypes

ID                           int64
name                        string
category                    string
main_category               string
currency                    string
deadline            datetime64[ns]
goal                       float64
launched            datetime64[ns]
pledged                    float64
state                       string
backers                    float64
country                     string
usd pledged                float64
usd_pledged_real           float64
dtype: object

In [120]:
# L'ensemble de données n'est pas forcément nécessaire donc nous allons passer en revue les questions de l'exercice pour supprimer les colonnes dont nous n'aurons pas besoin
df_ks = df_ks.drop(['currency', 'deadline', 'usd pledged', 'pledged', 'backers'], axis=1)
df_ks.columns

Index(['ID', 'name', 'category', 'main_category', 'goal', 'launched', 'state',
       'country', 'usd_pledged_real'],
      dtype='object')

In [121]:
df_ks.head()

,ID,name,category,main_category,goal,launched,state,country,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,2000.0,2011-08-17 06:31:31,canceled,US,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,18000.0,2012-07-23 20:46:48,failed,US,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,2000.0,2016-07-01 21:55:54,successful,US,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,13000.0,2017-08-28 15:47:02,successful,US,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,5000.0,2016-01-25 17:37:10,failed,CA,0.738225


### Importer les données

In [122]:
# Il conviendra de bien spécifier manuellement l'ID du document. 
df_ks['_id'] = df_ks['ID']
DOCUMENTS = df_ks.to_dict('records')
collection.drop()
collection.insert_many(DOCUMENTS)

## Question 1  

In [123]:
# On récupère les 5 projets ayant reçu le plus de promesse de dons.
cur = collection.find({}, {'name':1}).sort([("usd pledged", 1)]).limit(5)
list(cur)

[{'_id': 156812982,
  'name': 'SketchPlanner: Create and Plan- all in one beautiful book!'},
 {'_id': 1835968190,
  'name': 'Proven sales with custom motorcycle accessories'},
 {'_id': 1326492673, 'name': 'Ohceola jewelry'},
 {'_id': 872782264,
  'name': 'Scott Cooper\'s Solo CD "A Leg Trick" (Canceled)'},
 {'_id': 1688410639,
  'name': 'Sluff Off & Harald: Two latest EGGs are Classics "old & new"'}]

## Question 2

In [124]:
# On compte le nombre de projets ayant atteint leur but et qui sont donc "successful".
cur = collection.find( { "state": 'successful' })
len(list(cur))

52999

## Question 3

In [125]:
# On compte le nombre de projets pour chaque catégorie.
cur = collection.aggregate([{"$group" : {"_id" : "$main_category", "NumberByCategory" : {"$sum" : 1}}}])
list(cur)

[{'_id': 'Film & Video', 'NumberByCategory': 24982},
 {'_id': 'Journalism', 'NumberByCategory': 1807},
 {'_id': 'Photography', 'NumberByCategory': 4176},
 {'_id': 'Technology', 'NumberByCategory': 12871},
 {'_id': 'Art', 'NumberByCategory': 11260},
 {'_id': 'Crafts', 'NumberByCategory': 3463},
 {'_id': 'Publishing', 'NumberByCategory': 15721},
 {'_id': 'Music', 'NumberByCategory': 19658},
 {'_id': 'Food', 'NumberByCategory': 9805},
 {'_id': 'Games', 'NumberByCategory': 13987},
 {'_id': 'Fashion', 'NumberByCategory': 8943},
 {'_id': 'Theater', 'NumberByCategory': 4305},
 {'_id': 'Comics', 'NumberByCategory': 4139},
 {'_id': 'Dance', 'NumberByCategory': 1488},
 {'_id': 'Design', 'NumberByCategory': 11911}]

## Question 4

In [126]:
# On compte le nombre de projets français ("country": 'FR') ayant été instanciés avant 2016.
cur = collection.find({
     "country": 'FR' 
          ,
          'launched' : {'$lt' : datetime.datetime(2016, 1, 1)}})
len(list(cur))

330

## Question 5

In [127]:
# On récupère les projets américains ("country": 'US') ayant demandé plus de 200 000 dollars.
cur = collection.find({
     "country": 'US' 
          ,
          'goal' : {'$gt' : 20000}})
len(list(cur))

22196

## Question 6 

In [128]:
# On compte le nombre de projets ayant "Sport" dans leur nom.
collection.create_index([('name', 'text')])
cur = collection.find( { "$text": { "$search": "Sport" } } )
len(list(cur))

316